In [ ]:
import cv2 as cv
import numpy as np
from pyBEBLID import BEBLID_create

# Read the input images in grayscale format (cv.IMREAD_GRAYSCALE)
img1 = cv.imread('/home/gaditek/Receipt-Scanner/CustomerDataset/OPTP/Dollmen-Mall-Clifton/11798-638b75538e43b.png', 0)
img2 = cv.imread('/home/gaditek/Receipt-Scanner/ReceiptDataset/OPTP/optp-dollmen-mall-clifton-karachi/4fOzaLl0WJWWZFB18m6g9tlzgreMh67BlKDywAaV.png', 0)

# Create the feature detector, for example ORB
detector = cv.ORB_create()

# Detect features in both images
points1 = detector.detect(img1, None)
points2 = detector.detect(img2, None)
print("Detected", len(points1), "kps in image1")
print("Detected", len(points2), "kps in image2")

# Use 32 bytes per descriptor and configure the scale factor for ORB detector
descriptor = cv.BEBLID_create(256, 0.75)

# Describe the detected features in both images
_, descriptors1 = descriptor.compute(img1, points1)
_, descriptors2 = descriptor.compute(img2, points2)
print("Points described")

# Match the generated descriptors for img1 and img2 using brute force matching
matcher = cv.BFMatcher_create(cv.NORM_HAMMING, crossCheck=True)
matches = matcher.match(descriptors1, descriptors2)
print("Number of matches:", len(matches))

# If there are not enough matches exit
if len(matches) < 4:
    exit(-1)

# Take only the matched points that will be used to calculate the
# transformation between both images
matched_pts1 = np.array([points1[match.queryIdx].pt for match in matches])
matched_pts2 = np.array([points2[match.trainIdx].pt for match in matches])

# Find the homography that transforms a point in the first image to a point in the second image.
H, inliers = cv.findHomography(matched_pts1, matched_pts2, cv.RANSAC, 3)
# Print the number of inliers, that is, the number of points correctly
# mapped by the transformation that we have estimated
num_inliers = np.sum(inliers)
print("Number of inliers:", num_inliers, "(", 100.0 * num_inliers / len(matches), "% )")

# Convert the images to BGR format from grayscale
img1_bgr = cv.cvtColor(img1, cv.COLOR_GRAY2BGR)
img2_bgr = cv.cvtColor(img2, cv.COLOR_GRAY2BGR)

# Draw all the matched keypoints in red color
all_matches_img = cv.drawMatches(img1_bgr, points1, img2_bgr, points2, matches, None, (0, 0, 255), (0, 0, 255))

# Show and save the result
cv.imshow("All matches", all_matches_img)
cv.waitKey()
